In [1]:
import numpy as np
import math
from numpy.random import choice

In [2]:
x = [[0, 1], [1, 0]]
h = [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]
cx = [[0, 0], [0, 0]]#Initialised as zeros because it'll later be calculated using projection and x operator
# Define projection operator |0><0|
P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|
P1x1 = np.array([
[0, 0],
[0, 1]
])

In [10]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    v = []
    for i in range(2**num_qubits):
        if i == 0:
            v.append(1)
        else:
            v.append(0)
    return v

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    if gate_unitary == x or gate_unitary == h:
        #Code for single qubit gates
        I = np.identity(2)
        flag = 0
    
        if target_qubits[0] == 0:
            O = np.kron(gate_unitary, I)
            flag = 1
        elif target_qubits[0] == 1:
            O = np.kron(I, gate_unitary)
            flag = 1
    
        for i in range((total_qubits - 1) - flag):
            if flag == 1:
                O = np.kron(O, I)
            elif i == (target_qubits[0] - 1):
                O = np.kron(O, gate_unitary)
            elif i == 0:
                O = np.kron(I, I)
            else:
                O = np.kron(O, I)      
    
    elif gate_unitary == cx:
        #Code for multi-qubit gate cx
        I = np.identity(2)
        gate_unitary = x
        flag1 = 0
        flag2 = 0
    
        #Works for gate after projection PU or PI
        if target_qubits[0] == 0:
            if target_qubits[1] == 1:
                first_term = np.kron(P0x0, I)
                second_term = np.kron(P1x1, gate_unitary)
                flag2 = 1
                
            else:
                first_term = np.kron(P0x0, I)
                second_term = np.kron(P1x1, I)
            flag1 = 1
            
        elif target_qubits[0] == 1:
            if target_qubits[1] == 0:
                first_term = np.kron(I, P0x0)
                second_term = np.kron(gate_unitary, P1x1)
                flag2 = 1
            else:
                first_term = np.kron(I, P0x0)
                second_term = np.kron(I, P1x1)
            flag1 = 1
    
    
        for i in range((total_qubits - 1) - flag1):
            if flag1 == 1 and flag2 == 1:
                first_term = np.kron(first_term, I)
                second_term = np.kron(second_term, I)
            
            elif flag1 == 0 and i == (target_qubits[0] - 1):
                first_term = np.kron(first_term, P0x0)
                second_term = np.kron(second_term, P1x1)
                flag1 = 1
            
            elif flag2 == 0 and i == (target_qubits[1] - 2):
                first_term = np.kron(first_term, I)
                second_term = np.kron(second_term, gate_unitary)
                flag2 = 1        
        
            elif flag1 == 0 and flag2 == 0 and i == 0:
                first_term = np.kron(I, I)
                second_term = np.kron(I, I)
            
            else:
                first_term = np.kron(first_term, I)
                second_term = np.kron(second_term, I)
    
        O = first_term + second_term
    
    return O

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    final_state = initial_state
    for i in range(len(program)):
        O = get_operator(int(math.log(len(my_qpu))/math.log(2)), program[i]["gate"], program[i]["target"])
        print(O)
        final_state = np.dot(final_state, O)
    return final_state

def measure_all(state_vector, num_shots):
    # choose element from state_vector using weighted random and return it's index
    index = []
    for i in range(len(state_vector)):
        index.append(np.binary_repr(i, width = int(math.log(len(my_qpu))/math.log(2))))

    measurements = choice(index, num_shots, p = np.abs(state_vector)**2)

    return measurements

def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    results = measure_all(state_vector, num_shots)
    unique, counts = np.unique(results, return_counts=True)
    counts = dict(zip(unique, counts))
    return counts

In [11]:
# Create "quantum computer" with 2 qubits (this is actually just a vector :) )
my_qpu = get_ground_state(2)

my_qpu

[1, 0, 0, 0]

In [12]:
my_circuit = [
{ "gate": h, "target": [0] },
{ "gate": cx, "target": [0, 1]}
]

In [13]:
# Run circuit
final_state = run_program(my_qpu, my_circuit)
print("Final State: ",final_state)

[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678 -0.        ]
 [ 0.          0.70710678 -0.         -0.70710678]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
Final State:  [0.70710678 0.         0.         0.70710678]


In [14]:
Final = get_counts(final_state, 500)

In [15]:
Final

{'00': 247, '11': 253}